In [1]:
!git clone https://github.com/lucolivi/sentiment_analysis_dataset.git

Cloning into 'sentiment_analysis_dataset'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (9/9), 999.92 KiB | 11.63 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [2]:
from azure.ai.ml.entities import Data
from azure.identity import DefaultAzureCredential
from azure.ai.ml import automl, Input, MLClient, command
from azure.ai.ml.sweep import Choice

In [3]:
ml_client = MLClient.from_config(DefaultAzureCredential())

Found the config file in: /config.json


In [4]:
dados = Data(
    path="./sentiment_analysis_dataset/dataset.csv",
    type="uri_file",
    description="Conjunto de dados sobre sentimentos de textos em inglês.",
    name="dados-sentimentos",
)

ml_client.data.create_or_update(dados)

Data({'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'dados-sentimentos', 'description': 'Conjunto de dados sobre sentimentos de textos em inglês.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/507fb825-19a0-4a3f-8e46-2a5de5060508/resourceGroups/lucas.oliveira.fiap-rg/providers/Microsoft.MachineLearningServices/workspaces/meu_workspace_1/data/dados-sentimentos/versions/2', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/minhamaquina/code/Users/lucas.oliveira.fiap/hyperparam_tuning', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f09ad980d00>, 'serialize': <msrest.serialization.Serializer object at 0x7f09ad9808b0>, 'version': '2', 'latest_version': None, 'path': 'azureml://subscriptions/507fb825-19a0-4a3f-8e46-2a5de5060508/resourcegroups/l

In [5]:
job = command(
    inputs={
        "dataset": Input(type="uri_file", path="azureml:dados-sentimentos:1", mode="ro_mount"),
        "criterion": "gini",
        "n_estimators": 5
    },
    compute="meucluster",
    environment="AzureML-sklearn-1.0-ubuntu20.04-py38-cpu@latest",
    code="./",
    command="python analise_sent.py --dataset ${{inputs.dataset}} --criterion ${{inputs.criterion}} --n-estimators ${{inputs.n_estimators}}",
    experiment_name="analise_sentimentos",
    display_name="analise_sentimentos",
)

In [6]:
ml_client.jobs.create_or_update(job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading hyperparam_tuning (3.52 MBs)

In [7]:
job_for_sweep = job(
    criterion=Choice(values=["gini", "entropy"]),
    n_estimators=Choice(values=[5, 10, 15, 20, 25, 30])
)

In [8]:
sweep_job = job_for_sweep.sweep(
    compute="meucluster",
    sampling_algorithm="random",
    primary_metric="test_accuracy_score",
    goal="Maximize",
    max_total_trials=12,
    max_concurrent_trials=3,
)

In [9]:
ml_client.create_or_update(sweep_job)

Experiment,Name,Type,Status,Details Page
hyperparam_tuning,maroon_knee_rmt27n6chz,sweep,Running,Link to Azure Machine Learning studio
